In [ ]:
from pynq import Overlay
from pynq import DefaultIP
from pynq import PL
import random
import time
from threading import Thread

from pynq.lib.pmod import PMOD_GROVE_G3

from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

#Creating custom driver

class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        
    bindto = ["xilinx.com:user:LFSR_AXI_63BIT:1.0"]

    def set_reg0(self, Rst, Rst_Seed, En):
        self.write(0x00, Rst*1 + Rst_Seed*2 + En*4) #Sets Rst, Rst_Seed, En
        return
    
    def get_reg1(self):
        return self.read(0x04) #Returns LFSR Value
    
    def set_input(self, first32, last32):
        if (first32 > pow(2,32)-2) and (last32 > pow(2,32)-2) : #Avoid all 1s scenario to prevent lockup
            last32 = pow(2,32) - 4
        self.write(0x08, first32)
        self.write(0x0C, last32)
        return
    
ol = Overlay("design_1_wrapper.bit")
LFSR = ol.LFSR_AXI_64BIT_0     
        
pAudio = base.audio
pAudio.set_volume(40)

_playtune = False

def play_melody():
    # play melody
    global _playtune
    #pAudio.load("/home/xilinx/jupyter_notebooks/base/BingoProject/background.wav")
    pAudio.load("Audio_Files/background.wav")
    while _playtune == True:  
        pAudio.play()
        
###############################################################################################
        
def main():
    global _playtune
    
    #Initialisation
    LFSR.set_reg0(0,0,0) #Resets off, Enable on
    LFSR.set_input(0,0)
    
    Nvals = 90 #Sets number of values from 1 to 90
    valList = []
    outList = []
    
    #Create initial list of values
    for i in range(0,Nvals):
        valList.append(i + 1)

    remainingVals = Nvals
    selectedVal = 0
    randNum = 0
    numsInArray = 0
    
    userInput = ''
    endgame = False
    
    print("\nGame ready, press button 0 to begin")
    
    while True:
        if base.buttons[0].read()==1:
            print("\nOkay, let's begin")
            break

    while not endgame:
        
        #melody_thread = Thread(target=play_melody)
        #melody_thread.start()
            
        randNum = LFSR.get_reg1() #Gets LFSR random value (0 to 255)
        selectedVal = randNum % remainingVals #Selects number using modulus operator
            
        outList.append(valList[selectedVal]) #Stores selected value
        valList.pop(selectedVal) #Removes selected value from array
    
        remainingVals = remainingVals - 1
        numsInArray = numsInArray + 1     

        print("The number is ", outList[numsInArray - 1])
        print("\nTotal list of numbers called: ")
        for i in range(numsInArray):
            print(outList[i])
            
        #print("\nNvals = ",numsInArray)
        #print("\nRemaining vals = ", remainingVals)
        
        # Announce chosen value
        pAudio.load("Audio_Files/" + str(outList[numsInArray - 1]) + ".wav") #Last value in outlist
        pAudio.play()

        if numsInArray < 5:
            print("\nPlease Press Button 0 to continue :)")
        elif numsInArray >= 5 and numsInArray < 25:
            print("\nPlease Press Button 1 if you have a line, or Button 0 to continue :)")
        elif numsInArray >= Nvals:
            print("\nAll numbers have been called, the game is over")
            endgame = True
            break
        else:
            print("\nPlease Press Button 2 if you have Bingo, Button 1 if you have a line, or Button 0 to continue")

        while True:
            if base.buttons[3].read():
                print("Resetting game\n")
                endgame = True
                break
            
            if base.buttons[2].read():
                if numsInArray < 25:
                    print("This is impossible, not enough numbers have been called\n")
                    time.sleep(0.5)
                else:
                    print("\nBINGO! Congratulations, you win\nGame Over")
                    endgame = True
                    break
                          
            if base.buttons[1].read():
                if numsInArray < 5:
                    print("This is impossible, not enough numbers have been called\n")
                    time.sleep(0.5)
                else:
                    print("\nYou got a Line! Congratulations, now let's continue")
                    break
                    
            if base.buttons[0].read():
                print("\nOkay, let's continue")
                break
            
if __name__ == '__main__':
    from pynq.overlays.base import BaseOverlay
    base = BaseOverlay("base.bit")
    while True:
        main()


Game ready, press button 0 to begin

Okay, let's begin
The number is  46

Total list of numbers called: 
46

Please Press Button 0 to continue :)

Okay, let's continue
The number is  27

Total list of numbers called: 
46
27

Please Press Button 0 to continue :)

Okay, let's continue
The number is  45

Total list of numbers called: 
46
27
45

Please Press Button 0 to continue :)

Okay, let's continue
The number is  24

Total list of numbers called: 
46
27
45
24

Please Press Button 0 to continue :)

Okay, let's continue
The number is  38

Total list of numbers called: 
46
27
45
24
38

Please Press Button 1 if you have a line, or Button 0 to continue :)

Okay, let's continue
The number is  4

Total list of numbers called: 
46
27
45
24
38
4

Please Press Button 1 if you have a line, or Button 0 to continue :)

Okay, let's continue
The number is  3

Total list of numbers called: 
46
27
45
24
38
4
3

Please Press Button 1 if you have a line, or Button 0 to continue :)

Okay, let's continue